<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/main/ex07noteA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex07noteA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2022)


----
## 準備
----

Google Colab の Notebook では， Python というプログラミング言語のコードを動かして計算したりグラフを描いたりできます．
Python は，機械学習・人工知能やデータサイエンスの分野ではメジャーなプログラミング言語ですが，それを学ぶことはこの授業の守備範囲ではありません．以下の所々に現れるプログラムっぽい記述の内容は，理解できなくて構いません．

以下，コードセルを上から順に実行してながら読んでいってね．

In [ ]:
# 準備あれこれ
import numpy as np
import pandas as pd
from sklearn import tree # 機械学習ライブラリ scikit-learn の決定木パッケージ
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import seaborn
seaborn.set()

----
## 決定木
----






ニューラルネットは様々な問題で高い性能を発揮しますが，入力から出力に至る過程は，何をやっているのか人間が解釈するのが難しいブラックボックスとなりがちです．
正しい出力さえ得られればよい，汎化してくれればよいというなら,それでも構わないでしょうが，「どのようにしてその出力に至ったのか」を知りたい場合には困ります．
今回紹介する **決定木** (decision tree)は，そういう場合によく使われる学習の手法です．


---
### 例題: ラーメン？アイス？

決定木の説明のための例題を用意します．

In [ ]:
# データの入手
df = pd.read_csv('https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ramenice.csv')
df

このデータは，ほげおくんが $(気温, 湿度)$ のときに「アイスクリーム」を食べたか「ラーメン」を食べたかを記録したものです．$(気温, 湿度)$ を入力として「アイスクリーム」と「ラーメン」の2クラスに識別する問題と考えることができます．

散布図を描くと次のようになります．

In [ ]:
# データの準備
X = np.empty((len(df), 2))
X[:, 0] = df['気温'].to_numpy()
X[:, 1] = df['湿度'].to_numpy()
df['label'] = 0
df.loc[df['ラベル'] == 'アイスクリーム', 'label'] = 1
lab = df['label'].to_numpy()
print(X.shape, lab.shape)

# 散布図の描画
fig, ax = plt.subplots(facecolor="white", figsize=(8, 6))
ax.set_xlim(0, 40)
ax.set_ylim(0, 100)
ax.scatter(X[lab == 1, 0], X[lab == 1, 1], label='icecream') # blue
ax.scatter(X[lab == 0, 0], X[lab == 0, 1], label='ramen') # orange
ax.set_xlabel('Temparature', fontsize=16)
ax.set_ylabel('Humidity', fontsize=16)
ax.legend(fontsize=16)
plt.show()

---
### 決定木とは


**決定木**(Decision Tree) とは，その名の通り，木構造を利用した機械学習の仕組みです．識別にも回帰に用いることができますが，ここでは識別の例で説明します．

識別のための決定木は，下図左のように，葉以外のノード（節点，図の灰色の箱）にデータの値に関する条件が設定されており，葉ノード（オレンジ／青の箱）に識別対象のクラスが割り当てられた木です．
データが与えられたら，根ノードから子ノードを順次たどっていき，到達した葉ノードに割り当てられたクラスを出力します．

図の例で $(気温[度],湿度[\%]) = (15, 60)$ というデータを識別する場合，根ノードから左の子へとたどり，そこから右の子へとたどって「アイス」と識別されます．


<img width="100%" src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/decisiontree.png">




典型的な決定木は，上図左のように，ノードごとに入力データの変数のどれか一つ（例では「気温」か「湿度」のどちらか）に対する条件があり，データが条件を満たすか否かに応じた二つの子ノードを持つ二分木となっています（注）．

<span style="font-size: 75%">
※注: 決定木には様々なバリエーションがあり，複数の特徴量を組み合わせた条件を用いるものや，3つ以上の子に分岐するものなどもあります．
</span>

この図の決定木の例では，各ノードの条件によって入力の空間が上図右のように4つの領域に分けられ，それらが「アイス」「ラーメン」のどちらかに割り当てられることになります．


決定木では，学習データを用いて木構造を作ります．ノードごとの条件や分岐の仕方を学習データに応じて決めることになります．決定木の学習については後述することにして，以下では上記のデータを用いて実際に決定木を学習させてみます．

ここでは，[scikit-learn](https://scikit-learn.org/) という，Python による機械学習のためのライブラリを利用しています．



In [ ]:
# 決定木の学習
max_depth = 2
model = tree.DecisionTreeClassifier(max_depth=max_depth)
model = model.fit(X, lab) # 学習
pred = model.predict(X)  # 学習データの識別
ncorrect = np.sum(pred == lab)
N = X.shape[0]
print(f'木の深さの最大値: {max_depth}  学習データの識別率: {ncorrect}/{N} = {ncorrect/N}')

# 得られた決定木の描画
fig = plt.figure(facecolor='white', figsize=(8, 6))
ax = fig.add_subplot(111)
rv = tree.plot_tree(model, ax=ax, feature_names=['Temp', 'Hum'], class_names=['RAMEN', 'ICE'], filled=True)

上の図は，学習させて得られた決定木を可視化したものです．
それぞれの箱がノードであり，その中の一番上の行に条件が記されています．`Temp` は気温，`Hum` は湿度を表しています．
また，葉ノードの `class` はそのノードに割り当てられたクラスを表しています．箱の色については，次の節で説明します．

例えば，$(気温,湿度) = (28, 30)$ の場合，次のように木をたどって識別結果を得ることができます．

1. 根ノードの条件: `Hum <= 36.55` （$\mbox{(湿度)} \leq 36.55\mbox{[%]}$ ）を満たすので，左の子ノードへ
1. そのノードの条件: `Temp <= 22.6` （$\mbox{(気温)} \leq 22.6\mbox{[度]}$ ）を満たさないので，右の子ノードへ
1. 葉ノードへ到達．このノードは `class = ICE` なので，このデータは「アイス」と予測．


#### ★★ やってみよう ★★

上記のコードを実行して得られる決定木で，次のデータはどのように識別されるか考えて，紙媒体にメモしておきましょう．

- $(気温,湿度) = (26, 60)$
- $(気温,湿度) = (28, 38)$



以下に，上記の学習によって得られた決定木の作る識別境界を示します．この決定木では入力の特徴量ごとに条件を定めていますので，識別の境界は必ず入力空間のどれか一つの軸に平行となります．斜めになったり曲がったりすることはありません．

In [ ]:
# 識別境界の描画
fig = plt.figure(facecolor='white', figsize=(8, 6))
xm, ym = np.meshgrid(np.linspace(0, 40, num=100), np.linspace(0, 100, num=100))
Xm = np.vstack((xm.ravel(), ym.ravel())).T
pred = model.predict(Xm).reshape(xm.shape)
ax = fig.add_subplot(111)
ax.set_xlim(0, 40)
ax.set_ylim(0, 100)
ax.scatter(X[lab == 1, 0], X[lab == 1, 1], label='icecream') # blue
ax.scatter(X[lab == 0, 0], X[lab == 0, 1], label='ramen') # orange
ax.set_xlabel('Temparature', fontsize=16)
ax.set_ylabel('Humidity', fontsize=16)
ax.legend()
cmap = colors.ListedColormap(['orange','blue'])
ax.contourf(xm, ym, pred, cmap=cmap, alpha=0.2)
plt.show()

### 決定木の学習の考え方


決定木には様々なバリエーションがあります．ここではその代表的な学習法のひとつを説明します．

一般的に，決定木の学習では，根ノードに全ての学習データが割り当てられた状態からはじめて，学習データを二つに分けて二つの子ノードに割り振ることを繰り返して，木を成長させていきます．以下にその手続きの概略を示します．

1. 全ての学習データを根ノードに割り当てる．このノードを「現在のノード」とする．
1. 「現在のノード」に子ノードを作るかどうか判断する：
    - 子ノードを作る場合：現在のノードに割り当てられたデータを「最もうまく」二つに分ける条件を求める．二つの子ノードを生成し，その条件にしたがってデータを子ノードに割り振る．
    - 子ノードを作らない場合： 「現在のノード」に割り当てられたデータからこのノードのクラスを決める．
1. 子ノードを作った場合，左右の子ノードをあらためて「現在のノード」として，2. と 3. を繰り返す

ここで，

> 「現在のノード」に子ノードを作るかどうか判断する

ための基準としては，「そのノードに割り当てられるデータの数が一定以上になるようにする」，「木の深さ（根からそのノードまでにたどる枝の数）が一定以下になるようにする」，といったものが用いられます．
複雑すぎる決定木は過適合しやすくなりますので，決定木の学習ではこのような基準をうまく設定してやる必要があります．

また，ノードに学習データを振り分けていきますので，それらの所属クラスをなるべく偏らせて，葉ノードにはどれか一つのクラスのデータのみ含まれるようにするのが理想です．したがって，

> 現在のノードに割り当てられたデータを「最もうまく」二つに分ける条件を求める

の「最もうまく」の基準はそのように（ノードに振り分けられるデータの所属クラスがなるべく偏るように）定められます（下図参照）．

<img width="50%" src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/decisiontree2.png">

先の例題をもう一度使って学習の様子を見てみましょう．
実は，前述の学習の実験では木の深さの最大値を 2 に限定していました．これを変えてみましょう．



In [ ]:
#@title 決定木の学習
#@markdown 「ラーメン？アイス？」データの識別．
#@markdown 以下の `max_depth` は生成する決定木の深さの最大値を指定します．
max_depth =  2#@param [2, 3, 4, 5] {type: 'raw', allow-input: true}

# 決定木の学習
model = tree.DecisionTreeClassifier(max_depth=max_depth)
model = model.fit(X, lab) # 学習
pred = model.predict(X)  # 学習データの識別
ncorrect = np.sum(pred == lab)
N = X.shape[0]
print(f'木の深さの最大値: {max_depth}  学習データの識別率: {ncorrect}/{N} = {ncorrect/N}')

fig = plt.figure(facecolor='white', figsize=(8, 12))

# 得られた決定木の描画
ax0 = fig.add_subplot(211)
rv = tree.plot_tree(model, ax=ax0, feature_names=['Temp', 'Hum'], class_names=['RAMEN', 'ICE'], filled=True)

# 識別境界の描画
xm, ym = np.meshgrid(np.linspace(0, 40, num=100), np.linspace(0, 100, num=100))
Xm = np.vstack((xm.ravel(), ym.ravel())).T
pred = model.predict(Xm).reshape(xm.shape)
ax1 = fig.add_subplot(212)
ax1.set_xlim(0, 40)
ax1.set_ylim(0, 100)
ax1.scatter(X[lab == 1, 0], X[lab == 1, 1], label='icecream') # blue
ax1.scatter(X[lab == 0, 0], X[lab == 0, 1], label='ramen') # orange
ax1.set_xlabel('Temparature', fontsize=16)
ax1.set_ylabel('Humidity', fontsize=16)
ax1.legend()
cmap = colors.ListedColormap(['orange', 'blue'])
ax1.contourf(xm, ym, pred, cmap=cmap, alpha=0.2)

plt.show()

決定木を可視化した図は，次のようになっています．

- 各ノードの箱の中の `samples` の数は，そのノードに割り当てられた学習データの数を表す
- `value` の2つの数は，それらのデータのうち「アイス」クラスと「ラーメン」クラスに所属するものの数を表す
- 箱の色は，そのノードに所属するデータのクラスを表す（青がアイスでオレンジがラーメン）．色の濃さは，そのノードに属するデータの所属クラスの偏りに対応しており，一クラスのみだと濃く，混じっていると薄くなっている．

実験結果を観察すると，深いノードには少数の学習データしか割り当てられておらず，葉ノードに向かうにつれてクラスの偏りが大きくなっていることがわかります．
木の深さが小さく限定されていると葉ノードに2クラスのデータが混ざっていますが，木の深さが大きくなるとほとんどの葉ノードにどちらか一つのクラスのデータしかいなくなっています．

